In [1]:
#install chrome extension 'selectors hub' <https://chrome.google.com/webstore/detail/selectorshub/ndgimibanhlabgdgjcpbbndiehljcpfh/related?hl=en>
#this extension will make it easier to get xpath of dropdown list - relative and absolute
#select, enter, click etc are googled separately

#install selenium using foll command
#pip install selenium

In [2]:
from selenium import webdriver
from selenium.webdriver.common.alert import Alert           #to accept website alerts if required
from selenium.webdriver.support.select import Select        #to select a button or element
from selenium.webdriver.common.keys import Keys             #to type anything or hit ENTER
import time                                                 #to check loop runtime
from datetime import datetime                               #to get human timestamp in teh data for when data was captured
import os                                                   # for directory functions
import glob                                                 # for checking if file is downloaded
import numpy as np
import pandas as pd

In [117]:
###################################################################################################################
#                                            DEFINE EMPTY DATA FRAMES
###################################################################################################################

df_combined = pd.DataFrame(columns = ['S No', 'data_extrcn_date', 'data_extrcn_epoch', 'state', 'RTO', 'Maker',
       'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT',
       'NOV', 'DEC', 'TOTAL'])

#define the columns for a df_diagnostics which will get appended with the filters selected during each file download
df_diagnostics = pd.DataFrame(columns = ['StateLoopNum','Type','State','RTO','Y-axis','X-axis','Year Type', 'Year',
                                        'tickbox-selected','RowsNum','ColsNum','DownloadStatus','DownloadTimestamp',
                                         'DownloadCheckItrnCount','TimeTaken(mins)','driverNum','column_mismatch_error'])

#define a temp df to append current df summary data to the df_diagnostics
df_temp = pd.DataFrame({'StateLoopNum':[], 'Type':[], 'State':[], 'RTO':[], 'Y-axis':[], 'X-axis':[], 'Year Type':[],
       'Year':[], 'tickbox-selected':[],'RowsNum':[], 'ColsNum':[], 'DownloadStatus':[],'DownloadTimestamp':[],
       'DownloadCheckItrnCount':[],'TimeTaken(mins)':[],'driverNum':[],'column_mismatch_error':[]})

In [118]:
###################################################################################################################
#                                             DEFINE FUNCTIONS
###################################################################################################################

In [119]:
###*** function to read URL, and xpaths of all elements of the parivahan website from gsheet maintained separately
#this info is stored in a dataframe and returned to calling code
def gsheet_reader(url_passed):
    sheet_url = url_passed
    url_1 = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
    df_xpath=pd.read_csv(url_1,index_col=0)
    return (df_xpath)

In [120]:
###*** function to click on any drop down, click on required option in dropdown twice***################

#this process is done twice. hence the name filter_selector_2x2times
#xpaths of the dropdown click and the option to be selected are given by calling function
#if this function is successful the option selected in filter is returned
#need to implement return(0) if there is error in this function

def filter_selector_2x2times(driver, dropdown_xpath, option_xpath):
    driver.find_element("xpath", dropdown_xpath).click()      # click on the dropdown to expand it
    driver.find_element("xpath", option_xpath).click()          # click on the desired option name

    time.sleep(1)                                                                      # wait 1 sec before repeat click
    driver.find_element("xpath", option_xpath).click()          # click on the desired option name

    #repeating the process due to weird website behaviour
    driver.find_element("xpath", dropdown_xpath).click()      # click on the dropdown to expand it
    driver.find_element("xpath", option_xpath).click()          # click on the desired option name

    time.sleep(1)                                                                      # wait 1 sec before repeat click
    driver.find_element("xpath", option_xpath).click()          # click on the desired option name
    
    #get the name of the option selected
    filter_display = driver.find_element("xpath", dropdown_xpath).text
    
    return(filter_display)    

In [121]:
### function to click on the 'Year' drop down, click on 2022-23 in dropdown
def filter_selector_1x1times(driver, dropdown_xpath, option_xpath):
    driver.find_element("xpath", dropdown_xpath).click()         # click on the dropdown to expand it
    driver.find_element("xpath", option_xpath).click()             # click on the desired xaxis name
    time.sleep(1)                                                                       #wait 1 sec in case anything is running
    
    filter_display = driver.find_element("xpath", dropdown_xpath).text
    
    return(filter_display)

In [122]:
#function to get the display name of the filter on website
def filtered_selection(dropdown_xpath):
    filter_display = driver.find_element("xpath", dropdown_xpath).text
    return(filter_display)

In [123]:
#function to open website with required download directory and open the left tray

def website_opener(df_xpath):
    
    #change download folder
    options = webdriver.ChromeOptions()
    prefs = {"download.default_directory" : df_xpath['relXpath']['download folder link'],"directory_upgrade": True}   
    #extra \ in the download folder directory address, and 'upgrade' option is very imp
    
    options.add_experimental_option("prefs",prefs)

    #open chrome instance
    driver = webdriver.Chrome(executable_path='./chromedriver',options=options)

    # open the target URL given in xpaths dataframe
    driver.get(df_xpath['relXpath']['website link'])

    # Wait for 2 seconds to load the webpage completely
    time.sleep(2)
    
    # to check time when webdriver was opened.If website refreshes every 30mins, new driver will have to be opened before refresh
    driver_open_time = time.time()    

    #open the left filter tray 
    lefttray_xpath = df_xpath['absXpath']['left tray open button']
    driver.find_element("xpath", lefttray_xpath).click()
    time.sleep(1)                                             #wait 1 sec in case anything is running
    
    #select the correct options for y-axis=maker, x-axis=monthwise. 
    #yeartype cannot be changed when x axis is monthwise. year by default is cuurent year
    y_axis_selection = filter_selector_2x2times(driver,df_xpath['absXpath']['y-axis dropdown'],df_xpath['relXpath']['y-axis name'])
    x_axis_selection = filter_selector_2x2times(driver,df_xpath['absXpath']['x-axis dropdown'],df_xpath['relXpath']['x-axis name (monthwise)'])
    #yeartype_selection = filter_selector_2x2times(driver,df_xpath['absXpath']['year type dropdown'],df_xpath['relXpath']['year type name'])
    year_selection = filter_selector_1x1times(driver,df_xpath['relXpath']['year dropdown'],df_xpath['relXpath']['year name'])
    yeartype_selection = "Calendar Year"
    #year_selection = filtered_selection(df_xpath['relXpath']['year dropdown'])
    
    #pack selected filter display names in a list
    list_selections = ["dummyType","dummyState","dummyRTO",y_axis_selection,x_axis_selection,yeartype_selection,year_selection,
                       "dummyElectricSelected","dummyDownloadStatus","dummyDwnldChkRuns","DummyDwnldEPOCH",
                      "dummy2wICselected","dummy2wNTselected","dummy2wTselected"]
    
    print("website opened")                            #check how to handle failure msg

    return(driver,list_selections,driver_open_time)

In [124]:
#combined function to refresh the top panel, seletc left tray options, refresh it and downlaod generated file
###*** IMP FUNCTION TO DOWNLOAD THE XLS FILE **********************#####################

def refresh_n_downloader(driver,df_xpath,list_selections,check_itern_count=10):
    
    topright_refresh_xpath = df_xpath['absXpath']['topright refresh button']
    electric_checkbox_xpath = df_xpath['absXpath']['electric(BOV) checkbox']
    lefttray_refresh_xpath = df_xpath['absXpath']['left tray refresh button']
    TWOwhIC_checkbox_xpath = df_xpath['relXpath']['2WH(IC) checkbox']
    TWOwhNT_checkbox_xpath = df_xpath['relXpath']['2WH(NT) checkbox']
    TWOwhT_checkbox_xpath = df_xpath['relXpath']['2WH(T) checkbox']
    
    driver.find_element("xpath", topright_refresh_xpath).click()
    time.sleep(2)                                             #wait 1.5 sec in case anything is running

    #click on Fuel: ELectric(BOV) in teh left tray
    driver.find_element("xpath", electric_checkbox_xpath).click()
    time.sleep(1)                                             #wait 1 sec in case anything is running

    #click on 2wh IC, NT, T in teh left tray
    driver.find_element("xpath", TWOwhIC_checkbox_xpath).click()
    time.sleep(0.5)                                             #wait 0.3 sec in case anything is running
    driver.find_element("xpath", TWOwhNT_checkbox_xpath).click()
    time.sleep(0.5)                                             #wait 0.3 sec in case anything is running
    driver.find_element("xpath", TWOwhT_checkbox_xpath).click()
    time.sleep(0.5)                                             #wait 0.3 sec in case anything is running
    
    #click on the left tray refresh button
    driver.find_element("xpath", lefttray_refresh_xpath).click()
    time.sleep(2)                                             #wait 1 sec in case anything is running  

    #capture other remaining filter display items
    list_selections[0] = filtered_selection(df_xpath['absXpath']['Type dropdown'])
    list_selections[1] = filtered_selection(df_xpath['absXpath']['State dropdown'])
    list_selections[2] = filtered_selection(df_xpath['relXpath']['RTO dropdown'])
    list_selections[3] = filtered_selection(df_xpath['absXpath']['y-axis dropdown'])
    list_selections[4] = filtered_selection(df_xpath['absXpath']['x-axis dropdown'])
    #list_selections[5] = filtered_selection(df_xpath['absXpath']['year type dropdown'])
    list_selections[5] = "Calendar Year"
    list_selections[6] = filtered_selection(df_xpath['relXpath']['year dropdown'])
    list_selections[7] = not driver.find_element("xpath", electric_checkbox_xpath).is_selected()
    
    print(list_selections)
    
    # first count how many files you have in Downloads folder before download    
    time.sleep(1)                                             #wait 1 sec in case anything is running
    user = os.getlogin()
    downloads_folder = (df_xpath['relXpath']['download folder link'])                  #defined at the top for customizing webdriver session
    files_path = os.path.join(downloads_folder, '*')
    files = sorted(glob.iglob(files_path), key=os.path.getctime, reverse=True)
    files_before_download = files

    #print(f'files before download: {len(files)}')
    finished = False


    #click on the xlsheet download button************************************************************************   
    time.sleep(2)                                             #wait 1 sec in case anything is running
    driver.find_element("xpath", df_xpath['absXpath']['xlsheet download button']).click()
    #driver.find_element("xpath", df_xpath['absXpath']['xlsheet download button']).click()
    time.sleep(1)                                             #wait 1 sec in case anything is running
    #************************************************************************************************************

    # waiting and checking 10 times for the download to finish if there is +1 file in Downloads folder
    i = 0                                                               # counter to limit max iterations of loop
    while (not finished and i<check_itern_count):
        files = sorted(glob.iglob(files_path), key=os.path.getctime, reverse=True)
        #print('number of files in folder = ',len(files))
        if (len(files) == len(files_before_download)) or (len(files) == (len(files_before_download)+2)):
            i+=1
            #print('download not finished.','check loop = ',i,'/',check_itern_count,'\n')
            time.sleep(0.5)                                             #wait 0.5 sec in case anything is running
            finished = False 
        else:
            #print('download finished')
            finished = True

    last_downloaded_file = files[0]
    
    download_EPOCH = time.time()
    
    time.sleep(0.5)                                             #wait 0.5 sec in case anything is running
    #capture other items for list_selections
    list_selections[8] = finished
    list_selections[9] = i
    list_selections[10] = download_EPOCH
    
    return(list_selections)

In [125]:
####***function to read the downloaded excel and modify it to make a usable dataframe
####***then add it to a combined dataframe, and capture important parameters in diagonistic dataframe

def xlsheet_processor(download_folder_link,df_combined,df_temp, state_loop_counter,
                      list_selections,df_diagnostics,process_start_time,driver_count):
        
    #unpack the filter display names and other items in list_selections
    Type_selection = list_selections[0]
    #state_name = list_selections[1]
    try:
        state_name = list_selections[1].split("/")[0]+" a.k.a "+list_selections[1].split("/")[1]
    except:
        state_name = list_selections[1]
    try:
        RTO_selection = list_selections[2].split("/")[0]+" a.k.a "+list_selections[2].split("/")[1]
    except:
        RTO_selection = list_selections[2]
    y_axis_selection = list_selections[3]
    x_axis_selection = list_selections[4]
    yeartype_selection = list_selections[5]
    year_selection = list_selections[6]
    electric_tickbox_selection = list_selections[7]
    finished_status = list_selections[8]
    iterns_run = list_selections[9]
    download_EPOCH = list_selections[10]
    
    #folder_path = df_xpath['relXpath']['download folder link']
    folder_path = download_folder_link
    #print(folder_path)

    file_type = r'\*xlsx'
    files = glob.glob(folder_path + file_type)
    #print(files)

    #max_file = max(files, key=os.path.getctime)
    #print(max_file)

    #for the last downloaded file we have oldname and newname
    oldName = max(files, key=os.path.getctime)
    newName1 = os.path.join(folder_path, state_name+'_'+RTO_selection+'_EPOCH_'+str(round(download_EPOCH,0))+'.xlsx')

    # Rename the file
    os.rename(oldName, newName1)
    
    newName = newName1
    
    #read the excel
    df = pd.read_excel(newName,engine="openpyxl")
    #df.head()
    #df.shape

    #copying the headers to lower columns for sr.no, maker and total. 

    df.iloc[[2],[0]] = df.iloc[[0],[0]].copy()
    df.iloc[[2],[1]] = df.iloc[[0],[1]].copy()
    df.iloc[[2],[-1]] = df.iloc[[0],[-1]].copy()

    #drop the top blank rows and make 3rd row as the header
    df = df.drop(index = [0,1])
    new_header = df.iloc[0] #grab the first row for the header
    df = df[1:] #take the data less the header row
    df.columns = new_header #set the header row as the df header

    #add state column and move it to left after maker column
    df['state'] = state_name
    df['RTO'] = RTO_selection
    df['data_extrcn_date'] = datetime.today().strftime('%Y-%m-%d')
    df['data_extrcn_epoch'] = round(download_EPOCH,0)

    #popout column 'data_extraction_date' and move to 1st position : 0,1,2,3..
    column_to_move = df.pop("data_extrcn_date")
    df.insert(1, "data_extrcn_date", column_to_move)

    #popout column 'data_extraction_epoch' and move to 2nd position : 0,1,2,3..
    column_to_move = df.pop("data_extrcn_epoch")
    df.insert(2, "data_extrcn_epoch", column_to_move)

    #popout column 'state' and move to 3rd position : 0,1,2,3..
    column_to_move = df.pop("state")
    df.insert(3, "state", column_to_move)

    #popout column 'RTO' and move to 4th position : 0,1,2,3..
    column_to_move = df.pop("RTO")
    df.insert(4, "RTO", column_to_move)
        
    #rename the maker column for some reason it is different and creates issue during concat
    df.rename(columns = {'\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 Maker \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0':'Maker'}, inplace = True)
    df.rename(columns = {'\xa0\xa0\xa0\xa0\xa0TOTAL\xa0\xa0\xa0\xa0\xa0':'TOTAL'}, inplace = True)

    print(df.columns)
    
    #check if there is any mismatch in column count of df_combine and the downloaded file dataframe
    if(sum(df.columns == df_combined.columns) != df.shape[1]):
        #print("column count mismatch")
        column_mismatch_error = "ERROR"
    else:
        column_mismatch_error = "No Error"
    
    #capture process end time
    process_end_time = time.time()
    download_timestamp = datetime.now().strftime("%d-%m-%Y %H:%M:%S")
    
    #append the current dataframe to df_combined. This has all actual data
    df['data_extrcn_epoch'] = round(process_end_time,0)
    df_combined = pd.concat([df_combined,df])
    
    #putting all summary items in a temp df to append to df_diagnostics
    #filtered selection is a function to get the displayed name of any filter on the website
    df_temp.loc[0] = [state_loop_counter,Type_selection,state_name,RTO_selection,
                      y_axis_selection,x_axis_selection,yeartype_selection,year_selection,electric_tickbox_selection,
                      df.shape[0],df.shape[1],finished_status,download_timestamp,iterns_run,
                      round((process_end_time-process_start_time)/60,2),driver_count,column_mismatch_error]
    
    #append the current df summary data to the df_diagnostics dataframe. This has summary data for diagnostic
    df_diagnostics = pd.concat([df_diagnostics,df_temp])
    
    

    
    return(df_combined,df_diagnostics)

In [126]:
###################################################################################################################
#                                               ACTUAL PROCESS CODE
###################################################################################################################

In [127]:
#create the xpaths dataframe by reading googlesheet
df_xpath = gsheet_reader("https://docs.google.com/spreadsheets/d/1xGpaRkz6RqiuQAI84FIYzRD1NXX--6E4nKTowqRrdvY/edit#gid=0")
#df_xpath.head()

In [128]:
#open the website wih download folder change and left tray pulled out
(driver,list_selections,driver_open_time) = website_opener(df_xpath)
first_driver_open_time = driver_open_time                                  #for overall process time with multiple drivers

C:\Users\Pratik_soft\AppData\Local\Temp\ipykernel_11840\2183271839.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='./chromedriver',options=options)


website opened


##############***************###############****************#################**********
#         trial for RTO level download

#select a state
state_loop_counter = 8
filter_selector_2x2times(driver,df_xpath['absXpath']['State dropdown'],df_xpath['absXpath']['state'+str(state_loop_counter)])

#select a city from RTO dropdown 
rto_loop_counter = 2
filter_selector_1x1times(driver,df_xpath['relXpath']['RTO dropdown'],df_xpath['relXpath']['rto'+str(rto_loop_counter)])

In [129]:
driver_count = 1
df_drivers = pd.DataFrame(columns = ['Driver Number', 'driver open time', 'driver close time','run time (minutes)',
                                    'closing state','closing RTO'])
df_drivers

,Driver Number,driver open time,driver close time,run time (minutes),closing state,closing RTO


In [130]:

for state_loop_counter in range(1,35):
    

    ###########################################***STATE SELECTION***###########################################

    list_selections[1] = filter_selector_2x2times(driver,df_xpath['absXpath']['State dropdown'],
                                               df_xpath['absXpath']['state'+str(state_loop_counter)])

    
    rto_count = int(list_selections[1].split('(')[1].split(')')[0])
    print(list_selections[1]," has ",rto_count," rto offices")
    
    #click refresh as some RTOs with 0vehicles change format of the xlsx generated and distrub upcoming formulae
    driver.find_element("xpath", df_xpath['absXpath']['topright refresh button']).click()
    time.sleep(0.5)
    
    
    #for rto_loop_counter in range(1,3):                      #useful for testing out with limited downloads
    for rto_loop_counter in range(1,rto_count+1):
        process_start_time = time.time()                      # setting start time to check time of downloading 1 file
        
        list_selections[2] = filter_selector_1x1times(driver,df_xpath['relXpath']['RTO dropdown'],df_xpath['relXpath']['rto'+str(rto_loop_counter)])
    
        #refresh top panel, select left tray options, refresh left tray, download generated xl file
        list_selections = refresh_n_downloader(driver,df_xpath,list_selections,check_itern_count=10)

        #temp code to only click on download button and/or bypass rest of the loop
        #driver.find_element("xpath", df_xpath['absXpath']['xlsheet download button']).click()
        #print('file downloaded. bypass activated')
        #continue

        
        #skip xlsheet processing if donwload failed - else it might use previous iteration's downloaded file 
        if(not list_selections[8]):
            print("SKIPPING THE LOOP DUE TO DOWNLOAD FAILURE at ",list_selections[1].split('(')[0]," state ",
                 list_selections[2].split('(')[0]," rto")
            continue

        #read the downloaded file and delete top rows to make it usable
        (df_combined,df_diagnostics) = xlsheet_processor(df_xpath['relXpath']['download folder link'], 
                                                         df_combined, df_temp, state_loop_counter, 
                                                         list_selections,df_diagnostics,process_start_time,driver_count)
        
        if((time.time() - driver_open_time)/60 >10):
            driver.close()                                                              # close current driver to avoid website refresh interruption
            driver_close_time = time.time()                                             # to check time for which webdriver was open
            print("current driver closed. closed driver number = ",driver_count)
            df_drivers.loc[driver_count-1] = [driver_count,driver_open_time,driver_close_time,
                                 round((driver_close_time-driver_open_time)/60,2),
                                             list_selections[1],list_selections[2]]
            (driver,list_selections,driver_open_time) = website_opener(df_xpath)        #open new driver instance and select the state of current loop
            list_selections[1] = filter_selector_2x2times(driver,df_xpath['absXpath']['State dropdown'],
                                               df_xpath['absXpath']['state'+str(state_loop_counter)])
            #refresh the page as tabel format needs to change in a new driver after x,y axis selection
            driver.find_element("xpath", df_xpath['absXpath']['topright refresh button']).click()
            time.sleep(1.0)

            driver_count = driver_count+1
            print("new driver openend. opened driver number = ",driver_count)

West Bengal(55)  has  55  rto offices
['Actual Value', 'West Bengal(55)', 'SILIGURI ARTO - WB73( 30-MAR-2017 )', 'Maker', 'Month Wise', 'Calendar Year', '2022', True, 'dummyDownloadStatus', 'dummyDwnldChkRuns', 'DummyDwnldEPOCH', 'dummy2wICselected', 'dummy2wNTselected', 'dummy2wTselected']


C:\Users\Pratik_soft\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Index(['S No', 'data_extrcn_date', 'data_extrcn_epoch', 'state', 'RTO',
       'Maker', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP',
       'OCT', 'NOV', 'DEC', 'TOTAL'],
      dtype='object', name=2)
['Actual Value', 'West Bengal(55)', 'Srirampur ARTO - WB17( 10-JAN-2017 )', 'Maker', 'Month Wise', 'Calendar Year', '2022', True, True, 0, 1674110033.6824327, 'dummy2wICselected', 'dummy2wNTselected', 'dummy2wTselected']


C:\Users\Pratik_soft\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Index(['S No', 'data_extrcn_date', 'data_extrcn_epoch', 'state', 'RTO',
       'Maker', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP',
       'OCT', 'NOV', 'DEC', 'TOTAL'],
      dtype='object', name=2)
['Actual Value', 'West Bengal(55)', 'STA-Durgapur - WB997( 08-MAR-2019 )', 'Maker', 'Month Wise', 'Calendar Year', '2022', True, True, 0, 1674110046.997124, 'dummy2wICselected', 'dummy2wNTselected', 'dummy2wTselected']


...

C:\Users\Pratik_soft\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Index(['S No', 'data_extrcn_date', 'data_extrcn_epoch', 'state', 'RTO',
       'Maker', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP',
       'OCT', 'NOV', 'DEC', 'TOTAL'],
      dtype='object', name=2)
['Actual Value', 'West Bengal(55)', 'Uluberia ARTO - WB13( 25-JAN-2017 )', 'Maker', 'Month Wise', 'Calendar Year', '2022', True, True, 0, 1674110113.9725368, 'dummy2wICselected', 'dummy2wNTselected', 'dummy2wTselected']


C:\Users\Pratik_soft\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Index(['S No', 'data_extrcn_date', 'data_extrcn_epoch', 'state', 'RTO',
       'Maker', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP',
       'OCT', 'NOV', 'DEC', 'TOTAL'],
      dtype='object', name=2)
['Actual Value', 'West Bengal(55)', 'UTTAR DINAJPUR RTO - WB59( 04-APR-2017 )', 'Maker', 'Month Wise', 'Calendar Year', '2022', True, True, 0, 1674110127.3313055, 'dummy2wICselected', 'dummy2wNTselected', 'dummy2wTselected']
Index(['S No', 'data_extrcn_date', 'data_extrcn_epoch', 'state', 'RTO',
       'Maker', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP',
       'OCT', 'NOV', 'DEC', 'TOTAL'],
      dtype='object', name=2)


C:\Users\Pratik_soft\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [131]:
df_diagnostics

,StateLoopNum,Type,State,RTO,Y-axis,X-axis,Year Type,Year,tickbox-selected,RowsNum,ColsNum,DownloadStatus,DownloadTimestamp,DownloadCheckItrnCount,TimeTaken(mins),driverNum,column_mismatch_error
0,34,Actual Value,West Bengal(55),SILIGURI ARTO - WB73( 30-MAR-2017 ),Maker,Month Wise,Calendar Year,2022,True,9,19,True,19-01-2023 12:03:54,0,0.23,1,No Error
0,34,Actual Value,West Bengal(55),Srirampur ARTO - WB17( 10-JAN-2017 ),Maker,Month Wise,Calendar Year,2022,True,8,19,True,19-01-2023 12:04:07,0,0.22,1,No Error
0,34,Actual Value,West Bengal(55),STA-Durgapur - WB997( 08-MAR-2019 ),Maker,Month Wise,Calendar Year,2022,True,0,19,True,19-01-2023 12:04:20,0,0.22,1,No Error
0,34,Actual Value,West Bengal(55),STA-North Bengal (Siliguri) - WB998( 13-MAR-20...,Maker,Month Wise,Calendar Year,2022,True,0,19,True,19-01-2023 12:04:34,0,0.22,1,No Error
0,34,Actual Value,West Bengal(55),STA WEST BENGAL - WB999( 08-AUG-2018 ),Maker,Month Wise,Calendar Year,2022,True,0,19,True,19-01-2023 12:04:47,0,0.22,1,No Error
0,34,Actual Value,West Bengal(55),TAMLUK RTO - WB29( 09-FEB-2017 ),Maker,Month Wise,Calendar Year,2022,True,17,19,True,19-01-2023 12:05:01,0,0.23,1,No Error
0,34,Actual Value,West Bengal(55),Tehatta ARTO - WB52( 08-JUL-2020 ),Maker,Month Wise,Calendar Year,2022,True,2,19,True,19-01-2023 12:05:14,0,0.22,1,No Error
0,34,Actual Value,West Bengal(55),Uluberia ARTO - WB13( 25-JAN-2017 ),Maker,Month Wise,Calendar Year,2022,True,5,19,True,19-01-2023 12:05:27,0,0.22,1,No Error
0,34,Actual Value,West Bengal(55),UTTAR DINAJPUR RTO - WB59( 04-APR-2017 ),Maker,Month Wise,Calendar Year,2022,True,7,19,True,19-01-2023 12:05:41,0,0.22,1,No Error


In [132]:
if (df_diagnostics["RowsNum"].sum() == df_combined.shape[0]):
    print("all rows copied to df_combined")
else:
    print("ERROR: some rows not copied to df_combined")

all rows copied to df_combined


In [133]:
#save combined dataframe and diagnostic dataframe as csv
df_combined.to_csv(os.path.join(df_xpath['relXpath']['download folder link'],
                                'df_combined_EPOCH_'+str(round(time.time(),0))+'.csv'))
print("df_combined exported as csv")

df_diagnostics.to_csv(os.path.join(df_xpath['relXpath']['download folder link'],
                                   'df_diagnostics_EPOCH_'+str(round(time.time(),0))+'.csv'))
print("df_diagnostics exported as csv")

df_combined exported as csv
df_diagnostics exported as csv


In [134]:
# Close the driver
driver.close()
last_driver_close_time = time.time()    # to check time for which webdriver was open

In [135]:
driver_runtime = last_driver_close_time - first_driver_open_time

df_drivers.loc[driver_count-1] = [driver_count,driver_open_time,last_driver_close_time,
                                 round((last_driver_close_time-driver_open_time)/60,2),
                                  list_selections[1],list_selections[2]]

df_drivers

,Driver Number,driver open time,driver close time,run time (minutes),closing state,closing RTO
0,1,1.674110e+09,1.674110e+09,2.18,West Bengal(55),UTTAR DINAJPUR RTO - WB59( 04-APR-2017 )


In [136]:
print("full algo runtime = ",round(driver_runtime/60,1)," minutes")

try:
    print("in previous successful run this box was executed at ",last_execute_time)
except:
    print("this is the first run in this session")
last_execute_time = datetime.now().strftime("%d-%m-%Y %H:%M:%S")
print("in current  successful run this box was executed at",datetime.now().strftime("%d-%m-%Y %H:%M:%S"))

full algo runtime =  2.2  minutes
in previous successful run this box was executed at  19-01-2023 11:59:53
in current  successful run this box was executed at 19-01-2023 12:05:41
